In [1]:
#import the necessary libraries

import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import time

In [3]:
#read in the url from redfin
url = 'https://www.redfin.com/county/2918/UT/Utah-County'

In [4]:
#selenium code to navigate to the url in the chrome browser
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.get(url)

In [5]:
prices = []
beds = []
baths = []
square_footage = []
address = []
#code to account for the webdriver going through multiple pages of code
current_page = 1
pagination = driver.find_element(By.CLASS_NAME, 'PageNumbers')
last_page = pagination.find_elements(By.CLASS_NAME, 'ButtonLabel')[-1].text
while current_page <= int(last_page):
    #finds the element for the containers
    container = driver.find_element(By.CLASS_NAME, 'HomeCardsContainer')
    all_homes = container.find_elements(By.CLASS_NAME, 'HomeCardContainer')
    for home in all_homes:
        try:
            #finds the elements for each type of data we want to collect
            price = home.find_element(By.CLASS_NAME, 'bp-Homecard__Price--value').text
            bed = home.find_element(By.CLASS_NAME, 'bp-Homecard__Stats--beds').text
            bath = home.find_element(By.CLASS_NAME, 'bp-Homecard__Stats--baths').text
            sqft = home.find_element(By.CLASS_NAME, 'bp-Homecard__LockedStat--value').text
            add = home.find_element(By.CLASS_NAME, 'bp-Homecard__Address').text
            key = home.find_element(By.CLASS_NAME, 'KeyFactsExtension').text
        except:
            price = None
            bed = None
            bath = None
        #code to create lists for the data
        prices.append(price)
        beds.append(bed)
        baths.append(bath)
        square_footage.append(sqft)
        address.append(add)
    try:
        #code to find the button to click to the next page
        next_button = driver.find_element(By.CSS_SELECTOR, '.PageArrow__direction--next')
        next_button.click()
        current_page += 1
        #use a sleeper for ethical coding
        #time.sleep(2)
    except:
        pass

#we can quit the chrome driver once we have scraped our data
driver.quit()

In [ ]:
#create a dataframe with scraped data
df = pd.DataFrame({'Price': prices, 'Beds': beds, 'Baths': baths, 'Sq_Footage': square_footage, 'Address': address})
print(df.head(2))
print(df.tail(2))
df

In [ ]:
#create a csv of the dataframe
df.to_csv('redfin_data.csv')